In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Read the data into a dataframe
df = pd.read_csv("cleaned_data.csv")

# Separate the features and target variable
X = df[
    [
        "total_direct_mentions",
        "total_indirect_mentions",
        "total_likes",
        "total_retweets",
        "total_project_followers",
        "total_indirect_followers",
        "total_positive_direct_mentions",
        "total_negative_direct_mentions",
        "total_positive_indirect_mentions",
        "total_negative_indirect_mentions",
        "soft_cap",
    ]
]
y = df["ico_success"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Perform Random Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Define the parameter grids for grid search
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10],
    "min_samples_split": [2, 5, 10],
}

lr_param_grid = {"C": [0.1, 1, 10], "penalty": ["l1", "l2"]}

nb_param_grid = {}

svm_param_grid = {
    "C": [0.1, 1, 10],
    "gamma": [1e-2, 1e-3, 1e-4, 1e-5],
    "kernel": ["rbf"],
}

# Define the models
models = {
    "Naïve Bayes": (GaussianNB(), nb_param_grid),
    "SVM": (SVC(random_state=42), svm_param_grid),
    "Logistic Regression": (LogisticRegression(random_state=42), lr_param_grid),
    "Random Forest": (RandomForestClassifier(random_state=42), rf_param_grid),
}

result = {
    "Model": [],
    "Best Parameters": [],
    "Average Accuracy": [],
    "Average Precision": [],
    "Average Recall": [],
    "Average F1 Score": [],
}

best_estimators = {}

# Perform grid search and cross-validation for each model
for model_name, (model, param_grid) in models.items():
    print(f"Model: {model_name}")
    print("=" * 50)
    result["Model"].append(model_name)

    # Perform grid search with 5-fold cross-validation
    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=5, scoring="accuracy"
    )
    grid_search.fit(X_resampled, y_resampled)

    # Print the best parameters and the corresponding score
    print("Best Parameters: ", grid_search.best_params_)
    print()

    result["Best Parameters"].append(grid_search.best_params_)

    best_estimators[model_name] = grid_search.best_estimator_

    # Perform 5-fold cross-validation with the best model
    cv_results = cross_validate(
        grid_search.best_estimator_,
        X_resampled,
        y_resampled,
        cv=5,
        scoring=["accuracy", "precision", "recall", "f1"],
    )
    print("Cross-Validation Results:")
    print("=" * 50)
    overfitted_folds = 0  # Counter for overfitted folds
    for fold_idx, fold_result in enumerate(cv_results["test_accuracy"]):
        if fold_result == 1.0:  # Check for overfitted fold
            overfitted_folds += 1
            continue  # Skip overfitted fold

        print(f"Fold {fold_idx+1}:")
        print(f"Accuracy: {fold_result}")
        print(f"Precision: {cv_results['test_precision'][fold_idx]}")
        print(f"Recall: {cv_results['test_recall'][fold_idx]}")
        print(f"F1 Score: {cv_results['test_f1'][fold_idx]}")
        print()

    # Calculate average results across non-overfitted folds
    num_folds = len(cv_results["test_accuracy"])
    num_non_overfitted_folds = num_folds - overfitted_folds
    avg_accuracy = (
        sum(cv_results["test_accuracy"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_precision = (
        sum(cv_results["test_precision"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_recall = (
        sum(cv_results["test_recall"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds
    avg_f1 = (
        sum(cv_results["test_f1"]) - (overfitted_folds * 1)
    ) / num_non_overfitted_folds

    # Print the average results
    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1 Score: {avg_f1}")
    print()

    result["Average Accuracy"].append(avg_accuracy)
    result["Average Precision"].append(avg_precision)
    result["Average Recall"].append(avg_recall)
    result["Average F1 Score"].append(avg_f1)

result

Model: Naïve Bayes
Best Parameters:  {}

Cross-Validation Results:
Fold 1:
Accuracy: 0.45263157894736844
Precision: 0.3333333333333333
Recall: 0.09473684210526316
F1 Score: 0.14754098360655737

Fold 2:
Accuracy: 0.5
Precision: 0.5
Recall: 0.968421052631579
F1 Score: 0.6594982078853047

Fold 3:
Accuracy: 0.5105263157894737
Precision: 0.5055555555555555
Recall: 0.9578947368421052
F1 Score: 0.6618181818181819

Fold 4:
Accuracy: 0.4789473684210526
Precision: 0.4887640449438202
Recall: 0.9157894736842105
F1 Score: 0.6373626373626373

Fold 5:
Accuracy: 0.5105263157894737
Precision: 0.5061728395061729
Recall: 0.8631578947368421
F1 Score: 0.6381322957198443

Average Accuracy: 0.4905263157894737
Average Precision: 0.46676515466777635
Average Recall: 0.76
Average F1 Score: 0.548870461278505

Model: SVM
Best Parameters:  {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

Cross-Validation Results:
Fold 1:
Accuracy: 0.8473684210526315
Precision: 0.7661290322580645
Recall: 1.0
F1 Score: 0.867579908675799

Fo

/Users/inflaton/miniconda3/envs/ico/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/inflaton/miniconda3/envs/ico/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

Best Parameters:  {'C': 0.1, 'penalty': 'l2'}

Cross-Validation Results:
Fold 1:
Accuracy: 0.7
Precision: 0.6862745098039216
Recall: 0.7368421052631579
F1 Score: 0.7106598984771574

Fold 2:
Accuracy: 0.7105263157894737
Precision: 0.7702702702702703
Recall: 0.6
F1 Score: 0.6745562130177515

Fold 3:
Accuracy: 0.7263157894736842
Precision: 0.7792207792207793
Recall: 0.631578947368421
F1 Score: 0.6976744186046512

Fold 4:
Accuracy: 0.6368421052631579
Precision: 0.703125
Recall: 0.47368421052631576
F1 Score: 0.5660377358490566

Fold 5:
Accuracy: 0.6789473684210526
Precision: 0.7575757575757576
Recall: 0.5263157894736842
F1 Score: 0.6211180124223602

Average Accuracy: 0.6905263157894737
Average Precision: 0.7392932633741458
Average Recall: 0.5936842105263158
Average F1 Score: 0.6540092556741953

Model: Random Forest
Best Parameters:  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}

Cross-Validation Results:
Fold 1:
Accuracy: 0.8368421052631579
Precision: 0.8076923076923077
Rec

{'Model': ['Naïve Bayes', 'SVM', 'Logistic Regression', 'Random Forest'],
 'Best Parameters': [{},
  {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'},
  {'C': 0.1, 'penalty': 'l2'},
  {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}],
 'Average Accuracy': [0.4905263157894737,
  0.8684210526315788,
  0.6905263157894737,
  0.8978947368421052],
 'Average Precision': [0.46676515466777635,
  0.792547274749722,
  0.7392932633741458,
  0.9360105278308968],
 'Average Recall': [0.76, 1.0, 0.5936842105263158, 0.8610526315789475],
 'Average F1 Score': [0.548870461278505,
  0.884026921162544,
  0.6540092556741953,
  0.8951971505647952]}

In [2]:
from datasets import Dataset

perf_ds = Dataset.from_dict(result).to_pandas()

/Users/inflaton/miniconda3/envs/ico/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
perf_ds

,Model,Best Parameters,Average Accuracy,Average Precision,Average Recall,Average F1 Score
0,Naïve Bayes,"{'C': None, 'gamma': None, 'kernel': None, 'ma...",0.490526,0.466765,0.760000,0.548870
1,SVM,"{'C': 1.0, 'gamma': 0.01, 'kernel': 'rbf', 'ma...",0.868421,0.792547,1.000000,0.884027
2,Logistic Regression,"{'C': 0.1, 'gamma': None, 'kernel': None, 'max...",0.690526,0.739293,0.593684,0.654009
3,Random Forest,"{'C': None, 'gamma': None, 'kernel': None, 'ma...",0.897895,0.936011,0.861053,0.895197


In [4]:
res2 = perf_ds.drop(columns=["Best Parameters"])

In [5]:
res2 = res2.set_index(res2.columns[0]).mul(100)
res2

,Average Accuracy,Average Precision,Average Recall,Average F1 Score
Model,,,,
Naïve Bayes,49.052632,46.676515,76.000000,54.887046
SVM,86.842105,79.254727,100.000000,88.402692
Logistic Regression,69.052632,73.929326,59.368421,65.400926
Random Forest,89.789474,93.601053,86.105263,89.519715


In [6]:
for key in res2.select_dtypes(include=["number"]).columns:
    res2[key] = res2[key].apply("{:.1f}%".format)

res2

,Average Accuracy,Average Precision,Average Recall,Average F1 Score
Model,,,,
Naïve Bayes,49.1%,46.7%,76.0%,54.9%
SVM,86.8%,79.3%,100.0%,88.4%
Logistic Regression,69.1%,73.9%,59.4%,65.4%
Random Forest,89.8%,93.6%,86.1%,89.5%


In [7]:
best_estimators

{'Naïve Bayes': GaussianNB(),
 'SVM': SVC(C=1, gamma=0.01, random_state=42),
 'Logistic Regression': LogisticRegression(C=0.1, random_state=42),
 'Random Forest': RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)}

In [8]:
%%time
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

result = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1 Score": [],
}

for model_name in best_estimators:
    model = best_estimators[model_name]
    print(f"Model: {model_name}")
    print("=" * 50)
    result["Model"].append(model_name)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f_measure = f1_score(y_test, y_pred)

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F-measure:", f_measure)

    result["Accuracy"].append(accuracy)
    result["Precision"].append(precision)
    result["Recall"].append(recall)
    result["F1 Score"].append(f_measure)

result

Model: Naïve Bayes
Accuracy: 0.6402439024390244
Precision: 0.6518987341772152
Recall: 0.9626168224299065
F-measure: 0.7773584905660378
Model: SVM
Accuracy: 0.6524390243902439
Precision: 0.6524390243902439
Recall: 1.0
F-measure: 0.7896678966789668
Model: Logistic Regression
Accuracy: 0.6463414634146342
Precision: 0.855072463768116
Recall: 0.5514018691588785
F-measure: 0.6704545454545454
Model: Random Forest
Accuracy: 0.7804878048780488
Precision: 0.7933884297520661
Recall: 0.897196261682243
F-measure: 0.8421052631578947
CPU times: user 29.1 ms, sys: 2.58 ms, total: 31.7 ms
Wall time: 29.8 ms


{'Model': ['Naïve Bayes', 'SVM', 'Logistic Regression', 'Random Forest'],
 'Accuracy': [0.6402439024390244,
  0.6524390243902439,
  0.6463414634146342,
  0.7804878048780488],
 'Precision': [0.6518987341772152,
  0.6524390243902439,
  0.855072463768116,
  0.7933884297520661],
 'Recall': [0.9626168224299065, 1.0, 0.5514018691588785, 0.897196261682243],
 'F1 Score': [0.7773584905660378,
  0.7896678966789668,
  0.6704545454545454,
  0.8421052631578947]}

In [9]:
from datasets import Dataset

perf_ds = Dataset.from_dict(result).to_pandas()
perf_ds

,Model,Accuracy,Precision,Recall,F1 Score
0,Naïve Bayes,0.640244,0.651899,0.962617,0.777358
1,SVM,0.652439,0.652439,1.000000,0.789668
2,Logistic Regression,0.646341,0.855072,0.551402,0.670455
3,Random Forest,0.780488,0.793388,0.897196,0.842105


In [10]:
res2 = perf_ds.set_index(perf_ds.columns[0]).mul(100)
res2

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Naïve Bayes,64.024390,65.189873,96.261682,77.735849
SVM,65.243902,65.243902,100.000000,78.966790
Logistic Regression,64.634146,85.507246,55.140187,67.045455
Random Forest,78.048780,79.338843,89.719626,84.210526


In [11]:
for key in res2.select_dtypes(include=["number"]).columns:
    res2[key] = res2[key].apply("{:.1f}%".format)

res2

,Accuracy,Precision,Recall,F1 Score
Model,,,,
Naïve Bayes,64.0%,65.2%,96.3%,77.7%
SVM,65.2%,65.2%,100.0%,79.0%
Logistic Regression,64.6%,85.5%,55.1%,67.0%
Random Forest,78.0%,79.3%,89.7%,84.2%
